In [8]:
import pandas as pd

all_df = pd.read_csv("../data/all_data.csv")
site_df = pd.read_csv("../data/site_metadata.csv")

temp_df = all_df.merge(site_df, on=["CUSTOMER_NAME", "PLANT_NAME"])

all_df["THRM_EFF"] = temp_df["POWER"] / (temp_df["FUEL_FLOW"] * temp_df["FUEL_LHV"])

In [ ]:
import plotly.express as px
px.set_mapbox_access_token("pk.eyJ1IjoiYmVudGhlY29kZXIiLCJhIjoiY2wwNGNyb2ZzMGU4NjNjbnlzMXkxcmRnZiJ9.Jh6_T1hrlSoIjAD0UQG4HA")
fig = px.scatter_mapbox(site_df, lat="LATITUDE", lon="LONGITUDE", hover_name="CUSTOMER_NAME", hover_data=["CUSTOMER_NAME", "PLANT_NAME", "LATITUDE", "LONGITUDE"], height=1000, zoom=1)

fig.update_layout(
    title="Gas Turbine Locations",
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
)

fig.show()

In [ ]:
# group by customer name and plant name in all_df and calculate averages of each column
all_df.groupby(["CUSTOMER_NAME", "PLANT_NAME", "ENGINE_ID"]).mean().reset_index()
all_df.groupby(["CUSTOMER_NAME", "PLANT_NAME", "ENGINE_ID"]).std().reset_index()

In [ ]:
all_df.isnull().sum()

In [ ]:
# filter all_df by plant_name as SPIRITUAL-POLECAT

# all_df = all_df[all_df["PLANT_NAME"] == "SPIRITUAL-POLECAT"]

# plot heatmap of power overtime for each ENGINE_ID
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(all_df, x="datetime", y="POWER", color="ENGINE_ID", title="Power over time for each ENGINE_ID")

fig.show()

In [ ]:
# group by plant and sum power and average thrm_eff for each plant

power_df = all_df.groupby(["PLANT_NAME"])[["POWER", "THRM_EFF"]].agg({"POWER" : sum, "THRM_EFF" : "mean"}).reset_index()

# plot side by side bar chart of power and thrm_eff for each plant
power_df.sort_values(by=["THRM_EFF"], ascending=True, inplace=True)

power_df['THRM_EFF'] = power_df['THRM_EFF'] * 100

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=False,
                    shared_yaxes=True, horizontal_spacing=0)

fig.append_trace(go.Bar(x=power_df['POWER'],
                     y=power_df['PLANT_NAME'], 
                     text=power_df["POWER"].map('{:,.0f}'.format), #Display the numbers with thousands separators in hover-over tooltip 
                     textposition='inside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker_color='#4472c4'), 
                     1, 1) # 1,1 represents row 1 column 1 in the plot grid

fig.append_trace(go.Bar(x=power_df['THRM_EFF'],
                     y=power_df['PLANT_NAME'], 
                     text=power_df["THRM_EFF"].map('{:,.0f}'.format),
                     textposition='inside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker_color='#ed7d31'), 
                     1, 2) # 1,2 represents row 1 column 2 in the plot grid


fig.update_xaxes(showticklabels=False, title_text="Power Generation", row=1, col=1, range=[730000000,0])
fig.update_xaxes(showticklabels=False, title_text="Thermal Efficiency", row=1, col=2, range=[0, 50])

fig.update_layout(title_text="Power Generation", 
                  width=800, 
                  height=700,
                  xaxis1={'side': 'top'},
                  xaxis2={'side': 'top'},)

fig.show()


In [ ]:
# plot power against thermal efficiency for each plant
import plotly.express as px

fig = px.scatter(all_df, x="POWER", y="THRM_EFF", color="PLANT_NAME", hover_name="PLANT_NAME", title="Power vs Thermal Efficiency for each plant")
fig.show()

In [ ]:
all_df

In [ ]:
# convert df datatime to day of week
all_df["datetime"] = pd.to_datetime(all_df["datetime"])
all_df["DAYWEEK"] = all_df["datetime"].dt.dayofweek
all_df["TIME"] = all_df["datetime"].dt.strftime("%H:%M")
all_df

In [ ]:
all_df.groupby(["PLANT_NAME", "ENGINE_ID", "DAYWEEK", "TIME"])[['CMP_SPEED']] \
  .agg(lambda x: x.eq(0).sum()).reset_index()

avail_df = all_df.groupby(["PLANT_NAME", "ENGINE_ID"])[['CMP_SPEED']] \
  .agg(lambda x: x.eq(0).sum()).reset_index()

avail_df['CMP_SPEED'] = avail_df['CMP_SPEED'] / 7
avail_df

In [ ]:
# for each plant name, plot a horizontal grouped bar chart on CMP_SPEED
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=False,
                    shared_yaxes=True, horizontal_spacing=0)     

fig.append_trace(go.Bar(x=avail_df['PLANT_NAME'],
                      y=avail_df['CMP_SPEED'],      
                      text=avail_df["CMP_SPEED"].map('{:,.0f}'.format), #Display the numbers with thousands separators in hover-over tooltip
                      textposition='inside',
                      orientation='v',
                      width=0.7,
                      showlegend=False,
                      marker_color='#82DE94'),
                      1, 1) # 1,1 represents row 1 column 1 in the plot grid

fig.update_xaxes(showticklabels=False, title_text="Plant Name", row=1, col=1)
fig.update_xaxes(showticklabels=False, title_text="CMP_SPEED", row=1, col=2)

fig.update_layout(title_text="Average Downtime in Hours per week for each plant",
                  width=800,
                  height=700,
                  xaxis1={'side': 'top'},
                  xaxis2={'side': 'top'},)

fig.show()

In [ ]:
# for each plant plot a line graph to track the thrm_eff overtime for each engine_id and join empty time slots with a green line
import plotly.express as px
# get the subsection of the data where the plant name is SPIRITUAL-POLECAT
# get all the unique plat names
for plant in all_df["PLANT_NAME"].unique():
  if plant:
    current = all_df[all_df["PLANT_NAME"] == plant]
    fig = px.line(current, x="datetime", y="THRM_EFF", color="ENGINE_ID", title=f"Thermal Efficiency over time for each engine in {plant}")
    fig.show()




In [ ]:
# can't really sum by all plant since different plant has different usage of power
# plot the thrm_eff overtime for each engine_id and join empty time slots with a green line
import plotly.express as px

fig = px.line(all_df, x="datetime", y="THRM_EFF", color="ENGINE_ID", title="Thermal Efficiency over time for each engine")
fig.show()


In [ ]:
import pandas as pd
all_df = pd.read_parquet("https://hackathonfiles123.s3.amazonaws.com/all_df.parquet")

In [ ]:
all_df.to_parquet("../data/all_df.parquet")